# The Algorithm

In [20]:
import numpy as np
import pandas as pd
import xarray as xr
import hvplot.pandas  # noqa
import hvplot.xarray  # noqa
import cartopy.crs as ccrs
import random
import geoviews as gv
import holoviews as hv

from geopy.distance import geodesic

import warnings
import asyncio
warnings.filterwarnings('ignore')
pd.options.mode.chained_assignment = None
asyncio.set_event_loop_policy(asyncio.DefaultEventLoopPolicy())

A decentralized gready routing algorithm with memory to not to get stuck runing circles and maximum iterations of 200.

In [21]:
users = pd.read_csv(r'C:\Users\dajka\Documents\Egyetem\MSC\III\dsdatasci\data/us_users_in_network.csv')
edges_100 = pd.read_csv(r'C:\Users\dajka\Documents\Egyetem\MSC\III\dsdatasci\twitter/edges_1000.csv')
users_100 = users[users['user_id'].isin(np.unique(edges_100.values))].reset_index(drop=True)

In [22]:
def geodesic_distance(a, b):
    return geodesic(a, b).miles

In [23]:
def greedy_routing(users, edges, start, end):
    
    goal_loc = tuple(users[users['user_id']==end].iloc[:,1:3].values[0])
    goal_met = 0

    current_node = start
    current_loc = tuple(users[users['user_id']==start].iloc[:,1:3].values[0])
    current_dist = geodesic_distance(current_loc, goal_loc)
    iterations = 0
    problems = 0

    visited_nodes = []
    visited_locations_lat = []
    visited_locations_lon = []

    while goal_met < 1:

        iterations += 1
        visited_nodes.append(current_node)
        visited_locations_lat.append(current_loc[0])
        visited_locations_lon.append(current_loc[1])
        
        # Get the neighbors
        neighbors = np.unique(pd.concat([edges[edges['from']==current_node], edges[edges['to']==current_node]]).values).tolist()
        neighbors_df = users[users['user_id'].isin(neighbors)]

        # Calculate their distances from the goal
        distances = [geodesic_distance(tuple(row), goal_loc) for row in users[users['user_id'].isin(neighbors)].iloc[:,1:3].values]
        neighbors_df.loc[:,'distance'] = distances
        neighbors_df = neighbors_df[~neighbors_df['user_id'].isin(visited_nodes)] # neighbors not in visited nodes

        # Check if dead end
        if len(neighbors_df) < 1: 
            iterations = -1
            goal_met += 1
            break
        
        # Choose the colsest neighbor
        closest_neighbor = neighbors_df.sort_values('distance').iloc[0,0]

        if (neighbors_df.sort_values('distance').iloc[0,-1] == 0) and (closest_neighbor == end):
            visited_nodes.append(closest_neighbor)
            visited_locations_lat.append(users[users['user_id']==closest_neighbor].iloc[:,1].values[0])
            visited_locations_lon.append(users[users['user_id']==closest_neighbor].iloc[:,2].values[0])
            goal_met += 1
        elif neighbors_df.sort_values('distance').iloc[0,-1] >= current_dist:
            problems += 1

        # New current node
        current_node = closest_neighbor
        current_loc = tuple(users[users['user_id']==closest_neighbor].iloc[:,1:3].values[0])
        current_dist = geodesic_distance(current_loc, goal_loc)        
    
        # Max iterations
        if problems > 200:
            iterations = -2
            goal_met += 1
            
    return iterations, visited_nodes, visited_locations_lat, visited_locations_lon

In [24]:
hops, nodes, lat, lon = greedy_routing(users_100, edges_100, random.choice(users_100.user_id.values), random.choice(users_100.user_id.values))

## Plot trajectories

In [25]:
hops, nodes, lat, lon = greedy_routing(users_100, edges_100, random.choice(users_100.user_id.values), random.choice(users_100.user_id.values))
trajectory = pd.DataFrame(list(zip(nodes,lat,lon)), columns=(['user_id', 'Latitude', 'Longitude']))
trajectory.hvplot.paths('Longitude', 'Latitude', geo=True, color='red', alpha=1,
                       xlim=(-130, -70), ylim=(0, 55), tiles='ESRI')

:Overlay
   .Tiles.I :Tiles   [x,y]
   .Path.I  :Path   [Longitude,Latitude]

In [26]:
hops, nodes, lat, lon = greedy_routing(users_100, edges_100, random.choice(users_100.user_id.values), random.choice(users_100.user_id.values))
trajectory = pd.DataFrame(list(zip(nodes,lat,lon)), columns=(['user_id', 'Latitude', 'Longitude']))
trajectory.hvplot.paths('Longitude', 'Latitude', geo=True, color='red', alpha=1,
                       xlim=(-130, -70), ylim=(0, 55), tiles='ESRI')

:Overlay
   .Tiles.I :Tiles   [x,y]
   .Path.I  :Path   [Longitude,Latitude]

In [27]:
hops, nodes, lat, lon = greedy_routing(users_100, edges_100, random.choice(users_100.user_id.values), random.choice(users_100.user_id.values))
trajectory = pd.DataFrame(list(zip(nodes,lat,lon)), columns=(['user_id', 'Latitude', 'Longitude']))
trajectory.hvplot.paths('Longitude', 'Latitude', geo=True, color='red', alpha=1,
                       xlim=(-130, -70), ylim=(0, 55), tiles='ESRI')

:Overlay
   .Tiles.I :Tiles   [x,y]
   .Path.I  :Path   [Longitude,Latitude]

In [28]:
hops, nodes, lat, lon = greedy_routing(users_100, edges_100, random.choice(users_100.user_id.values), random.choice(users_100.user_id.values))
trajectory = pd.DataFrame(list(zip(nodes,lat,lon)), columns=(['user_id', 'Latitude', 'Longitude']))
trajectory.hvplot.paths('Longitude', 'Latitude', geo=True, color='red', alpha=1,
                       xlim=(-130, -70), ylim=(0, 55), tiles='ESRI')

:Overlay
   .Tiles.I :Tiles   [x,y]
   .Path.I  :Path   [Longitude,Latitude]